<a href="https://colab.research.google.com/github/tyler55792/Ammonia-sequential-modular-simulation/blob/main/Ammonia_Process_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import copy
import numpy as np

class Stream:
  def __init__(self, T, P, molFlow_H, molFlow_N, molFlow_NH):
    self.T = T          # C
    self.P = P          # bar
    self.molFlow_H = molFlow_H        # kmol/hr
    self.molFlow_N = molFlow_N        # kmol/hr
    self.molFlow_NH = molFlow_NH      # kmol/hr
    self.molFlow_total = self.molFlow_H + self.molFlow_N + self.molFlow_NH

  def __str__(self):
    return f" Temp(C): {self.T:.2f}, Pressure(bar): {self.P:.2f}, Molar Flow Rate(kmol/hr): {self.molFlow_total:.2f}, H: {self.molFlow_H:.2f}, N: {self.molFlow_N:.2f}, NH: {self.molFlow_NH:.2f}"

def compressor(feed, pressure):   # outlet presssure in bar
  outlet = copy.copy(feed)
  outlet.T= ((outlet.T + 273.15) * np.power(200.5 / outlet.P, (1.4 - 1) / 1.4)) - 273.15
  outlet.P = pressure             # set compressor outlet stream to the specified pressure in bar
  return outlet

def mixer(feed1, feed2):
  outlet = copy.copy(feed1)
  outlet.molFlow_H = feed1.molFlow_H + feed2.molFlow_H
  outlet.molFlow_N = feed1.molFlow_N + feed2.molFlow_N
  outlet.molFlow_NH = feed1.molFlow_NH + feed2.molFlow_NH
  outlet.molFlow_total = feed1.molFlow_total + feed2.molFlow_total
  outlet.T = (feed1.T * feed1.molFlow_total + feed2.T * feed2.molFlow_total)/(feed1.molFlow_total + feed2.molFlow_total)
  return outlet

def heat(feed, temperature):
  outlet = copy.copy(feed)
  outlet.T = temperature          # set stream to specified temperature
  outlet.P = float(outlet.P) - 1    # pressure drop of 1 bar in heat exchanger
  return outlet

def reactor(feed):
  extent = feed.molFlow_N * 0.25
  outlet = copy.copy(feed)
  outlet.molFlow_N = outlet.molFlow_N - extent
  outlet.molFlow_H = outlet.molFlow_H - (3 * extent)
  outlet.molFlow_NH = outlet.molFlow_NH + (2 * extent)
  outlet.molFlow_total = outlet.molFlow_total - (2 * extent)
  return outlet

def valve(feed, pressure):
  outlet = copy.copy(feed)
  outlet.P = pressure             # set valve outlet stream to the specified pressure in bar
  return outlet

def flash(feed):
  components = ["Hydrogen", "Nitrogen", "Ammonia"]
  f = np.array([feed.molFlow_H, feed.molFlow_N, feed.molFlow_NH])  # in kmol/hr
  A = np.array([13.6333, 14.9342, 16.9481])
  B = np.array([164.9, 588.72, 2132.5])
  C = np.array([3.19, -6.6, -32.98])
  NC = len(components)  # number of components

  # Create variables.
  P0 = np.zeros(NC)     # vapor pressures (in mmHg)
  alpha = np.zeros(NC)  # relative volatilities w.r.t. to the key component
  K = np.zeros(NC)      # K values
  xi = np.zeros(NC)     # split fractions
  v = np.zeros(NC)      # vapor molar flowrates (in kmol/hr)
  l = np.zeros(NC)      # liquid molar flowrates (in kmol/hr)
  y = np.zeros(NC)      # vapor mole fractions
  x = np.zeros(NC)      # liquid mole fractions

  n=0     #hydrogen chosen as key component

  # Specified:
  P = feed.P * 750.062 # convert bar to mmHg
  T = feed.T + 273.15  # convert C to K

  # Guess split fraction of ammonia 0.95 at 0.2:
  xi[n] = 0.95

  while True:
    # Compute vapor pressures using the Antoine equation.
    P0 = np.exp(A - B / (T + C))  # in mmHg

    # Compute relative volatilities.
    alpha = P0 / P0[n]

    # Calculate recoveries.
    xi = alpha * xi[n] / (1 + (alpha - 1) * xi[n])

    # Solve mass balance and calculate mole fractions.
    v = xi * f
    l = (1 - xi) * f
    y = v / sum(v)
    x = l / sum(l)

    #Compute the average volatility. Close enough? If not, reguess xi_n
    k = 0
    alpha_avg = sum(alpha * x)
    alpha_avg_comp = P * alpha[k] / P0[k]  # from bubble point equation

    if abs(alpha_avg_comp - alpha_avg) < 0.01:
      print('flash converged')
      break

    xi[n] += 0.0001        #increment xi[n]

    if xi[n] > 1:
      print('Did not converge')

  # Now set molar outlet flows using split fractions
  outlet_vap = copy.copy(feed)   # vapor outlet
  outlet_liq = copy.copy(feed)   # liquid outlet

  outlet_vap.molFlow_H = outlet_vap.molFlow_H * xi[0]
  outlet_vap.molFlow_N = outlet_vap.molFlow_N * xi[1]
  outlet_vap.molFlow_NH = outlet_vap.molFlow_NH * xi[2]
  outlet_vap.molFlow_total = outlet_vap.molFlow_H + outlet_vap.molFlow_N + outlet_vap.molFlow_NH

  outlet_liq.molFlow_H = outlet_liq.molFlow_H * (1 - xi[0])
  outlet_liq.molFlow_N = outlet_liq.molFlow_N * (1 - xi[1])
  outlet_liq.molFlow_NH = outlet_liq.molFlow_NH * (1 - xi[2])
  outlet_liq.molFlow_total = outlet_liq.molFlow_H + outlet_liq.molFlow_N + outlet_liq.molFlow_NH

  return outlet_vap, outlet_liq

def split(feed, fraction):
  recycle = copy.copy(feed)
  purge = copy.copy(feed)

  recycle.molFlow_H = (1 - fraction) * feed.molFlow_H
  recycle.molFlow_N = (1 - fraction) * feed.molFlow_N
  recycle.molFlow_NH = (1 - fraction) * feed.molFlow_NH
  recycle.molFlow_total = (1 - fraction) * feed.molFlow_total

  purge.molFlow_H = (fraction) * feed.molFlow_H
  purge.molFlow_N = (fraction) * feed.molFlow_N
  purge.molFlow_NH = (fraction) * feed.molFlow_NH
  purge.molFlow_total = (fraction) * feed.molFlow_total

  return recycle, purge


In [12]:
if __name__ == '__main__':
  feed = Stream(25, 1.013, 75, 25, 0)     # create feed stream
  C1_1 = compressor(feed, 201)            # feed through compressor 1
  C2_1 = Stream(211, 201, 140, 40, 70)    # Guess flow rates

  #155, 50, 70
  i = 1
  while True:
    M1_1 = mixer(C1_1, C2_1)                # mole balance with mixer function
    E1_1 = heat(M1_1, 425)                  # set to reactor conditions
    R1_1 = reactor(E1_1)                    # reaction
    V1_1 = valve(R1_1, 51)                  # pressure drop through valve to 51 bar
    E2_1 = heat(V1_1, 25)                   # heat to 25 C and pressure drop 1 bar
    F1_1, F1_2 = flash(E2_1)
    S1_1, S1_2 = split(F1_1, 0.1)           # 10% split fraction

    print('Iteration: ' + str(i));
    print('C1_1:' + str(C1_1))
    print('C2_1:' + str(C2_1))
    print('M1_1:' + str(M1_1))
    print('E1_1:' + str(E1_1))
    print('R1_1:' + str(R1_1))
    print('V1_1:' + str(V1_1))
    print('E2_1:' + str(E2_1))
    print('S1_1:' + str(S1_1))
    print('S1_2:' + str(S1_2))
    print('F1_1:' + str(F1_1))
    print('F1_2:' + str(F1_2))
    print()
    i = i + 1

    #Tare
    if abs(S1_1.molFlow_H - C2_1.molFlow_H) < 1 and abs(S1_1.molFlow_N - C2_1.molFlow_N) < 1 and abs(S1_1.molFlow_NH - C2_1.molFlow_NH) < 1:
      print('Tare Converged')
      break

    C2_1.molFlow_H = S1_1.molFlow_H
    C2_1.molFlow_N = S1_1.molFlow_N
    C2_1.molFlow_NH = S1_1.molFlow_NH
    C2_1.molFlow_total = C2_1.molFlow_H + C2_1.molFlow_N + C2_1.molFlow_NH

  print('Final Solution:')
  print('C1_1:' + str(C1_1))
  print('C2_1:' + str(C2_1))
  print('M1_1:' + str(M1_1))
  print('E1_1:' + str(E1_1))
  print('R1_1:' + str(R1_1))
  print('V1_1:' + str(V1_1))
  print('E2_1:' + str(E2_1))
  print('S1_1:' + str(S1_1))
  print('S1_2:' + str(S1_2))
  print('F1_1:' + str(F1_1))
  print('F1_2:' + str(F1_2))



flash converged
Iteration: 1
C1_1: Temp(C): 1077.61, Pressure(bar): 201.00, Molar Flow Rate(kmol/hr): 100.00, H: 75.00, N: 25.00, NH: 0.00
C2_1: Temp(C): 211.00, Pressure(bar): 201.00, Molar Flow Rate(kmol/hr): 250.00, H: 140.00, N: 40.00, NH: 70.00
M1_1: Temp(C): 458.60, Pressure(bar): 201.00, Molar Flow Rate(kmol/hr): 350.00, H: 215.00, N: 65.00, NH: 70.00
E1_1: Temp(C): 425.00, Pressure(bar): 200.00, Molar Flow Rate(kmol/hr): 350.00, H: 215.00, N: 65.00, NH: 70.00
R1_1: Temp(C): 425.00, Pressure(bar): 200.00, Molar Flow Rate(kmol/hr): 317.50, H: 166.25, N: 48.75, NH: 102.50
V1_1: Temp(C): 425.00, Pressure(bar): 51.00, Molar Flow Rate(kmol/hr): 317.50, H: 166.25, N: 48.75, NH: 102.50
E2_1: Temp(C): 25.00, Pressure(bar): 50.00, Molar Flow Rate(kmol/hr): 317.50, H: 166.25, N: 48.75, NH: 102.50
S1_1: Temp(C): 25.00, Pressure(bar): 50.00, Molar Flow Rate(kmol/hr): 225.08, H: 146.17, N: 42.68, NH: 36.24
S1_2: Temp(C): 25.00, Pressure(bar): 50.00, Molar Flow Rate(kmol/hr): 25.01, H: 16.24,